In [1]:
# Application of FL task
from MLModel import *
from FLModel import *
from utils import *

from torchvision import datasets, transforms
import torch
import numpy as np
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "3"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
def load_data():
    # data sets
    data = []
    for i in range(1, 7):
        # d = np.load("/home/jyfan/data/bank/non-iid/3clients/bank" + str(i) + ".npy")
        d = np.load("/home/jyfan/data/bank/non-iid/bank" + str(i) + ".npy")
        data.append((d[:, :16], d[:, 16:].flatten()))
    return data


def load_mnist(num_users):
    # data_train = datasets.MNIST(root="~/data/", train=True, transform=transforms.ToTensor())
    data_train = datasets.MNIST(root="~/data/", train=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

    # data_test = datasets.MNIST(root="~/data/", train=False, transform=transforms.ToTensor())
    data_test = datasets.MNIST(root="~/data/", train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

    # split MNIST (training set) into non-iid data sets
    non_iid = []
    '''
    for i in range(0, 10):
        idx = np.where(data_train.targets == i)
        d = data_train.data[idx].flatten(1).float()
        targets = data_train.targets[idx].float()
        non_iid.append((d, targets))
    non_iid.append((data_test.data.flatten(1).float(), data_test.targets.float()))
    '''
    user_dict = mnist_noniid(data_train, num_users)
    for i in range(num_users):
        idx = user_dict[i]
        d = data_train.data[idx].flatten(1).float()
        targets = data_train.targets[idx].float()
        non_iid.append((d, targets))
    non_iid.append((data_test.data.flatten(1).float(), data_test.targets.float()))
    return non_iid


def load_p(latent):
    pth = "/home/jyfan/data/MNIST/"
    non_iid_p = []
    for i in range(10):
        d = np.load(pth + "non-iid-p/" + str(latent) + "/P_" + str(i) + ".npy")
        d = (d.T / abs(d).max(1)).T
        target = [i*1.0 for ii in range(d.shape[0])]
        non_iid_p.append((np.array(d), np.array(target)))
    test = np.load(pth + "non-iid-p/" + str(latent) + "/P10_test.npy")
    test = (test.T / abs(test).max(1)).T
    t_label = np.load(pth + "non-iid-p/" + str(latent) + "/test_label.npy")
    non_iid_p.append((test, t_label))
    return non_iid_p

In [3]:
"""
1. load_data
2. generate clients (step 3)
3. generate aggregator
4. training
"""
# d = load_data()
client_num = 10
d = load_mnist(client_num)
#d = load_p(latent=10)

In [72]:
fl_par = {
    'output_size': 10,
    'client_num': client_num,
    'model': MLP,
    'data': d,
    'lr': 0.001,
    'E': 5,
    'C': 1,
    'epsilon': 1.0,
    'delta': 1e-4,
    'clip': 200,
    'tot_E': 10,
    'batch_size': 32,
    'device': device
}
import warnings
warnings.filterwarnings("ignore")
fl_entity = FLServer(fl_par).to(device)
#fl_entity.global_update()

In [73]:
for name, parms in fl_entity.clients[1].model.named_parameters(): #.grad.state_dict()
    print('-->name:', name, '-->grad_requirs:',parms.requires_grad, ' -->grad_value:',parms.grad)

-->name: model.0.weight -->grad_requirs: True  -->grad_value: None
-->name: model.0.bias -->grad_requirs: True  -->grad_value: None
-->name: model.3.weight -->grad_requirs: True  -->grad_value: None
-->name: model.3.bias -->grad_requirs: True  -->grad_value: None


In [74]:
#fl_entity.global_update()

In [75]:
fl_entity.global_update_grad()

global epochs = 1, acc = 0.2070
global epochs = 2, acc = 0.3379
global epochs = 3, acc = 0.4186
global epochs = 4, acc = 0.4773
global epochs = 5, acc = 0.5120
global epochs = 6, acc = 0.5475
global epochs = 7, acc = 0.5668
global epochs = 8, acc = 0.5970
global epochs = 9, acc = 0.6202
global epochs = 10, acc = 0.6371


In [15]:
#print(fl_entity.clients[1].state_dict()['model.model.0.weight'].grad)
#print(fl_entity.clients[0].state_dict()['model.model.0.bias'].grad)
import copy
d=dict(fl_entity.clients[0].model.named_parameters())
grad = {}
for name in d:
    grad[name] = copy.deepcopy(d[name].grad)
    print(grad[name].shape)

d=dict(fl_entity.clients[1].model.named_parameters())
grad2 = {}
for name in d:
    grad2[name] = copy.deepcopy(d[name].grad)
#x=dict(fl_entity.clients[1].model.named_parameters())
#x['model.0.weight']
#for name, parms in fl_entity.clients[1].model.named_parameters(): #.grad.state_dict()
    #print('-->name:', name, '-->grad_requirs:',parms.requires_grad, ' -->grad_value:',parms.grad)
    #input()

torch.Size([200, 784])
torch.Size([200])
torch.Size([10, 200])
torch.Size([10])
